In [1]:
import os
os.chdir('..')

import json
from apps.calcs import *
from apps.data import player_json_to_df
import plotly.graph_objects as go

In [2]:
with open('C:/Users/jrnas/Downloads/BBGM_League_2_2024_preseason.json', encoding='latin') as f:
    r_json = json.load(f)

df = player_json_to_df(r_json)
df = df[df.season == df[~df.salary.isna()].season.max()].drop_duplicates(['pid', 'season']).reset_index(drop=True)

In [5]:
def load_and_process_data(json_file):
    r_json = json.load(json_file)
    df = player_json_to_df(r_json)
    df = df[df.season == df[~df.salary.isna()].season.max()].drop_duplicates(['pid', 'season']).reset_index(drop=True)
    df['results'] = df.apply(lambda x: calc_progs(x['ovr'], x['age'], 0.75), axis=1)
    df['rating_prog'] = df['results'].apply(lambda x: x['rating'])
    df['rating_upper_prog'] = df['results'].apply(lambda x: x['rating_upper'])
    df['rating_lower_prog'] = df['results'].apply(lambda x: x['rating_lower'])
    df['vorp_added_prog'] = df['results'].apply(lambda x: x['vorp_added'])
    df['cap_value_prog'] = df['results'].apply(lambda x: x['cap_value'])
    df['team'] = df['tid'].map(dict([(teams['tid'], teams['abbrev']) for teams in r_json['teams']]))
    return df

In [6]:
df = load_and_process_data(open('C:/Users/jrnas/Downloads/BBGM_League_2_2024_preseason.json', encoding='latin'))

In [7]:
df.head()

,pid,firstName,lastName,season,hgt,stre,spd,jmp,endu,ins,...,vorp,ewa,salary,results,rating_prog,rating_upper_prog,rating_lower_prog,vorp_added_prog,cap_value_prog,team
0,0,Evan,Mobley,2024,67,58,55,61,70,68,...,0.0,0.0,10000,"{'rating': {0: 64, 1: 64.98245998684497, 2: 66...","{0: 64, 1: 64.98245998684497, 2: 66.0349549467...","{0: 64, 1: 68.30430430430431, 2: 70.3063063063...","{0: 64, 1: 61.097097097097105, 2: 61.297297297...","{0: 4.275118756882797, 1: 4.711477446348202, 2...","{0: 0.29580138084432844, 1: 0.3259936422123854...",CLE
1,1,Julius,Randle,2024,56,50,62,45,99,70,...,0.0,0.0,30300,"{'rating': {0: 65, 1: 62.62057108685903, 2: 59...","{0: 65, 1: 62.62057108685903, 2: 59.9480301657...","{0: 65, 1: 64.8998998998999, 2: 62.89789789789...","{0: 65, 1: 60.295295295295304, 2: 56.891891891...","{0: 4.7189779381015775, 1: 3.664402594549525, ...","{0: 0.3265126116127297, 1: 0.25354508472827564...",NYK
2,2,Rudy,Gobert,2024,80,96,35,54,70,69,...,0.0,0.0,43800,"{'rating': {0: 66, 1: 62.3418872499495, 2: 58....","{0: 66, 1: 62.3418872499495, 2: 58.74028069657...","{0: 66, 1: 64.6986986986987, 2: 61.69569569569...","{0: 66, 1: 59.8938938938939, 2: 55.68968968968...","{0: 5.162837119320354, 1: 3.5398025869945005, ...","{0: 0.35722384238113064, 1: 0.2449238378380806...",MIN
3,3,Myles,Turner,2024,70,60,44,70,70,46,...,0.0,0.0,20500,"{'rating': {0: 65, 1: 63.197065820777155, 2: 6...","{0: 65, 1: 63.197065820777155, 2: 60.917737007...","{0: 65, 1: 65.50050050050051, 2: 63.8988988988...","{0: 65, 1: 60.895895895895904, 2: 57.892892892...","{0: 4.7189779381015775, 1: 3.9189532011985255,...","{0: 0.3265126116127297, 1: 0.2711577933390742,...",IND
4,5,Draymond,Green,2024,55,74,35,44,25,60,...,0.0,0.0,25000,"{'rating': {0: 60, 1: 55.24398073836275, 2: 50...","{0: 60, 1: 55.24398073836275, 2: 50.1473896619...","{0: 60, 1: 57.297297297297305, 2: 52.892892892...","{0: 60, 1: 53.0930930930931, 2: 47.28728728728...","{0: 2.49968203200768, 1: 0.8390973725568016, 2...","{0: 0.17295645777072374, 1: 0.0580583079863094...",GSW


In [11]:
def player_plot(pid, df):
    
    # Filter the dataframe for the specific pid
    df_filtered = df[df.pid == pid]
    
    # Retrieve the player's first name and last name
    first_name = df_filtered['firstName'].values[0]
    last_name = df_filtered['lastName'].values[0]
    
    # Create the title
    title = f'{first_name} {last_name}'
    
    # Extract the rating, value, and bounds data
    rating_data = df_filtered['rating_prog'].values[0]
    rating_upper_data = df_filtered['rating_upper_prog'].values[0]
    rating_lower_data = df_filtered['rating_lower_prog'].values[0]
    value_data = df_filtered['vorp_added_prog'].values[0]
    cap_value_data = df_filtered['cap_value_prog'].values[0]
    
    # Create the plot for ratings
    fig = go.Figure()
    
    # Add the plot for upper and lower bounds as lines with the area between them filled
    fig.add_trace(
        go.Scatter(
            x=list(rating_upper_data.keys()),
            y=list(rating_upper_data.values()),
            name='Upper Bound',
            mode='lines',
            line=dict(width=0),
            hoverinfo='skip',
            showlegend=False,
        )
    )
    
    fig.add_trace(
        go.Scatter(
            x=list(rating_lower_data.keys()),
            y=list(rating_lower_data.values()),
            name='Lower Bound',
            mode='none',
            fill='tonexty',
            hoverinfo='skip',
            fillcolor='rgba(255, 90, 95, 0.2)',
            showlegend=False,
        )
    )
    
    # Add the plot for ratings as a line
    fig.add_trace(
        go.Scatter(
            x=list(rating_data.keys()),
            y=list(rating_data.values()),
            name='Rating',
            line=dict(
                color='rgb(255, 90, 95)'
            ),
            customdata=np.stack((list(rating_lower_data.values()), list(rating_upper_data.values())), axis=-1),
            hovertemplate=
            '<b>Year</b>: %{x}<br>' +
            '<b>Rating</b>: %{y:.1f}<br>' +  # Round to 1 decimal place
            '<b>Lower Bound</b>: %{customdata[0]:.1f}<br>' +  # Round to 1 decimal place
            '<b>Upper Bound</b>: %{customdata[1]:.1f}<br>',  # Round to 1 decimal place
        )
    )
    
    # Add the plot for values as bars
    fig.add_trace(
        go.Bar(
            x=[x - 0.2 for x in list(value_data.keys())],
            y=list(value_data.values()),
            name='Value',
            yaxis='y2',
            width=0.35,
            marker=dict(
                color='rgb(252,100,45)'
            ),
            text=[f'{val:.1f}' for val in list(value_data.values())],
            textposition='outside',
            textfont=dict(
                color='rgb(252,100,45)',
                size=14,
            ),
        )
    )
    
    # Add the plot for cap values as bars
    fig.add_trace(
        go.Bar(
            x=[x + 0.2 for x in list(cap_value_data.keys())],
            y=list(cap_value_data.values()),
            name='Cap Value',
            yaxis='y3',
            width=0.35,
            marker=dict(
                color='rgb(0, 166, 153)'
            ),
            text=[f'{val*100:.1f}%' for val in list(cap_value_data.values())],
            textposition='outside',
            textfont=dict(
                color='rgb(0, 166, 153)',
                size=14,
            ),
        )
    )
    
    # Update the plot title
    fig.update_layout(
        template='simple_white',
        title=title,
        barmode='group',
        yaxis=dict(
            range=[10, 90],
            showgrid=False,
            showticklabels=True,
        ),
        yaxis2=dict(
            range=[0, 20],
            overlaying='y',
            side='right',
            showgrid=False,
            showticklabels=False,
        ),
        yaxis3=dict(
            range=[0, 1],
            overlaying='y',
            side='left',
            showgrid=False,
            showticklabels=False,
        )
    )
    return fig

In [13]:
player_plot(int(input('Enter a pid: ')), df)